<a href="https://colab.research.google.com/github/evinai/CLI-Ar-Todo-23/blob/master/2_5_Practice_Fundamentals_Most_Basic_Form_of_Training_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practice Fundamentals: Most Basic Form of Training LLMs 💪

## Learning Objectives 🎯
- Set up the development environment to utilize GPU resources.
- Understand and install specific library versions directly from a repository.
- Familiarize with YAML configuration for training setups.
- Execute a basic training session for a language model using the Axolotl library.

## GPU Verification ✅
Check for the availability of a GPU. A free-tier T4 GPU is sufficient for running this notebook, which makes it accessible without needing high-end hardware.

In [ ]:
!pip install torch==2.4.0

/bin/bash: line 1: conda: command not found


In [1]:
import torch
# Check so there is a gpu available, a T4(free tier) is enough to run this notebook
assert (torch.cuda.is_available()==True)

## Library Installation 🛠️
Install the Axolotl library directly from GitHub to ensure compatibility with the course's specified version. This step ensures that the environment matches the course requirements without needing advanced hardware.

In [2]:
!pip install -e 'git+https://github.com/axolotl-ai-cloud/axolotl.git@78b42a3fe13c49e317bc116b9999c30e070322cc#egg=axolotl' # ensures the same version we used in the course

Obtaining axolotl from git+https://github.com/axolotl-ai-cloud/axolotl.git@78b42a3fe13c49e317bc116b9999c30e070322cc#egg=axolotl
  Cloning https://github.com/axolotl-ai-cloud/axolotl.git (to revision 78b42a3fe13c49e317bc116b9999c30e070322cc) to ./src/axolotl
  Running command git clone --filter=blob:none --quiet https://github.com/axolotl-ai-cloud/axolotl.git /content/src/axolotl
  Running command git rev-parse -q --verify 'sha^78b42a3fe13c49e317bc116b9999c30e070322cc'
  Running command git fetch -q https://github.com/axolotl-ai-cloud/axolotl.git 78b42a3fe13c49e317bc116b9999c30e070322cc
  Running command git checkout -q 78b42a3fe13c49e317bc116b9999c30e070322cc
  Resolved https://github.com/axolotl-ai-cloud/axolotl.git to commit 78b42a3fe13c49e317bc116b9999c30e070322cc
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/huggingface/transformers.git (to revision 026a173a64372e9602a16523b8fae9de4b0ff428) to /tmp/pip-install-8hh7intt/transformers_2bc2970b6141491c81a115551b

In [3]:
!pip3 install --no-build-isolation axolotl[flash-attn,deepspeed]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.4 MB/s eta 0:00:00
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 58.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 77.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 84.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached ninja-1.11.1.3-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.3 kB

## Configuration Setup 📝
Create a YAML configuration to meticulously set up the training parameters. This configuration file will include settings for the model, tokenizer, and training details, structured to work efficiently even on less powerful, free-tier GPUs.

In [3]:
import yaml

train_config = """
# model params
base_model: TinyLlama/TinyLlama-1.1B-Chat-v1.0
model_type: LlamaForCausalLM
tokenizer_type: LlamaTokenizer


# dataset params
datasets:
  - path: jaydenccc/AI_Storyteller_Dataset
    type:
      system_prompt: ""
      field_system: system
      field_instruction: synopsis
      field_output: short_story
      format: "<|user|>\n {instruction} </s>\n<|assistant|>"
      no_input_format: "<|user|> {instruction} </s>\n<|assistant|>"

output_dir: ./models/evinai_TinyLlama_Storyteller

# model params
sequence_length: 1024
bf16: auto
tf32: false

# training params
batch_size: 4
micro_batch_size: 4
num_epochs: 2
optimizer: adamw_bnb_8bit
learning_rate: 0.0002

logging_steps: 1
"""

# Convert the YAML string to a Python dictionary
yaml_dict = yaml.safe_load(train_config)


# Write the YAML file
with open("basic_train.yml", 'w') as file:
    yaml.dump(yaml_dict, file)


## Training Launch 🚀
Launch the training process with the `accelerate` command. This command is optimized for use even with free-tier resources, ensuring that you can train models effectively without requiring premium hardware.

In [4]:
!accelerate launch -m axolotl.cli.train basic_train.yml

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2024-12-27 14:54:49.328493: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-27 14:54:49.346147: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-27 14:54:49.366959: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cu

In [12]:
# Install transformers from source - only needed for versions <= v4.34
# pip install git+https://github.com/huggingface/transformers.git
# pip install accelerate

import torch
from transformers import pipeline

pipe = pipeline("text-generation", model="/content/models/evinai_TinyLlama_Storyteller", torch_dtype=torch.bfloat16, device_map="auto")

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    # {
    #     "role": "system",
    #     "content": "You are a friendly chatbot who always responds in the style of a pirate",
    # },
    {"role": "user", "content": "A bright sudent was working with the Fuzzy cientist on a project."},
]


Device set to use cuda:0


In [13]:
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)



In [7]:
prompt

'<|user|>\na bright sudent was working with the Fuzzy scientist on a project.</s>\n<|assistant|>\n'

In [14]:
outputs = pipe(prompt, max_new_tokens=512)

In [15]:
print(outputs[0]["generated_text"])

<|user|>
A bright sudent was working with the Fuzzy cientist on a project.</s>
<|assistant|>
Armed with a bachelor's degree in cognitive science, Jack had always felt like there was something missing from his life. He had always been interested in the study of mental health, but he never imagined what it would be like for him to develop a cure for a high-profile researcher.

One day, as he sat at his desk, Jack heard a knock on his door. The knock was on his door and there was no name on it. It was unclear who was calling for help, so he opened the door to find a curious researcher named Dr. Jameson.

"Can I help you?" Jack asked, kindly.

Dr. Jameson replied that he could help but his expertise was limited. "I've got everything I need to get you the cure. The science is in the lab," he said, hoping that that was enough to make it go through to Jack.

"Ok, Jack. I've got this. The scientist and the engineer have both agreed to work with you in the lab, and they'll put up with any cost,

In [16]:
outputs = pipe(prompt, max_new_tokens=512)
print(outputs[0]["generated_text"])

<|user|>
A bright sudent was working with the Fuzzy cientist on a project.</s>
<|assistant|>
Dr. Alexis wasn't one for a spotlight, but he had to do his job. He sat in a small laboratory, working with the fuzzy cientist, who was determined to make a cure for his cancer patient.

The researchers were working tirelessly, analyzing the data from the patient's bloodstream and the survival patterns of the various tumors that had been used as a test. They were determined to find a cure that was both simple and easy to use.

The doctor started by analyzing the data from the patient's blood and the survival patterns of the cancer tissues. The results of the review were staggering. It was found that the cancer patient was in a much better condition than had been initially thought, and the drug could help him to live longer.

The doctor knew the stakes were high, but the only way to judge the effectiveness of the drug was to subject it to a rigorous test that was done on lab mice. He spent weeks

In [21]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) hf_JwWLyEjuqOXHsgKzGruNiuHvjqyKLLyb

In [20]:
!git config --global credential.helper store


In [22]:
!huggingface-cli repo create tiny_llama_story_v1

git version 2.34.1
git-lfs/3.0.2 (GitHub; linux amd64; go 1.18.1)

You are about to create evinai/tiny_llama_story_v1
Proceed? [Y/n] y

Your repo now lives at:
  https://huggingface.co/evinai/tiny_llama_story_v1

You can clone it locally with the command below, and commit/push as usual.

  git clone https://huggingface.co/evinai/tiny_llama_story_v1

